# Library imports

In [1]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.2 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import xlsxwriter as xl
import requests as rq
from scipy import stats
import math

# Importing Our List Of Stocks

In [56]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [57]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


#API Calling

In [7]:
!pip install pandas==1.3.3 pandas-datareader==0.9.0

In [8]:
import pandas_datareader as pdr
import datetime

In [9]:
symbol = 'AAPL'

In [10]:
import yfinance as yf

symbol = 'AAPL'
ticker = yf.Ticker(symbol)

# Fetching the general information and historical data
info = ticker.info
history = ticker.history(period="1y")

# Constructing a similar structure to the IEX Cloud API output
data = {
    'symbol': symbol,
    'companyName': info.get('shortName', None),
    'primaryExchange': info.get('exchange', None),
    'calculationPrice': 'latestClose',  # Closest concept in Yahoo Finance
    'open': info.get('regularMarketOpen', None),
    'openTime': None,  # Yahoo Finance does not provide exact time
    'openSource': None,
    'close': info.get('regularMarketPreviousClose', None),
    'closeTime': None,  # Not available in Yahoo Finance
    'closeSource': None,
    'high': info.get('dayHigh', None),
    'highTime': None,  # Not available in Yahoo Finance
    'highSource': None,
    'low': info.get('dayLow', None),
    'lowTime': None,  # Not available in Yahoo Finance
    'lowSource': None,
    'latestPrice': info.get('regularMarketPreviousClose', None),
    'latestSource': 'Yahoo Finance Data',
    'latestTime': None,  # Yahoo Finance does not provide exact time
    'latestUpdate': None,  # Timestamp not directly available
    'latestVolume': info.get('regularMarketVolume', None),
    'iexRealtimePrice': None,  # Not applicable in Yahoo Finance
    'iexRealtimeSize': None,
    'iexLastUpdated': None,
    'delayedPrice': None,  # Not applicable in Yahoo Finance
    'delayedPriceTime': None,
    'oddLotDelayedPrice': None,
    'oddLotDelayedPriceTime': None,
    'extendedPrice': info.get('postMarketPrice', None),
    'extendedChange': None,
    'extendedChangePercent': None,
    'extendedPriceTime': None,
    'previousClose': info.get('regularMarketPreviousClose', None),
    'previousVolume': info.get('volume', None),
    'change': info.get('regularMarketChange', None),
    'changePercent': info.get('regularMarketChangePercent', None),
    'volume': info.get('volume', None),
    'iexMarketPercent': None,  # Not available in Yahoo Finance
    'iexVolume': None,
    'avgTotalVolume': info.get('averageVolume', None),
    'iexBidPrice': None,
    'iexBidSize': None,
    'iexAskPrice': None,
    'iexAskSize': None,
    'iexOpen': None,
    'iexOpenTime': None,
    'iexClose': None,
    'iexCloseTime': None,
    'marketCap': info.get('marketCap', None),
    'peRatio': info.get('trailingPE', None),
    'week52High': info.get('fiftyTwoWeekHigh', None),
    'week52Low': info.get('fiftyTwoWeekLow', None),
    'ytdChange': None,  # Calculate manually if needed
    'lastTradeTime': None,  # Yahoo Finance does not provide this
    'isUSMarketOpen': info.get('marketState', None) == 'REGULAR',
}

data


{'symbol': 'AAPL',
 'companyName': 'Apple Inc.',
 'primaryExchange': 'NMS',
 'calculationPrice': 'latestClose',
 'open': 244.05,
 'openTime': None,
 'openSource': None,
 'close': 243.01,
 'closeTime': None,
 'closeSource': None,
 'high': 244.54,
 'highTime': None,
 'highSource': None,
 'low': 242.13,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 243.01,
 'latestSource': 'Yahoo Finance Data',
 'latestTime': None,
 'latestUpdate': None,
 'latestVolume': 36910248,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 243.01,
 'previousVolume': 36910248,
 'change': None,
 'changePercent': None,
 'volume': 36910248,
 'iexMarketPercent': None,
 'iexVolume': None,
 'avgTotalVolume': 48495584,
 'iexBidPrice': None,
 'iexBidSize

In [11]:
price = data['latestPrice']
price

243.01

In [12]:
pe_ration = data['peRatio']
pe_ration

39.973682

#Executing A Batch API Call & Building Our DataFrame


In [59]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [60]:
# import yfinance as yf
# import pandas as pd

# # split the list into chunks
# def chunks(lst, n):
#     """Yield successive n-sized chunks from lst."""
#     for i in range(0, len(lst), n):
#         yield lst[i:i + n]

# stocks = pd.read_csv('sp_500_stocks.csv')

# # Split the stock tickers into groups of 100
# symbol_groups = list(chunks(stocks['Ticker'], 100))
# symbol_strings = [','.join(group) for group in symbol_groups]

# # Initialize an empty DataFrame
# final_dataframe = pd.DataFrame(columns=my_columns)

# # Loop through each batch of symbols
# for symbols in symbol_groups:
#     for symbol in symbols:
#         try:
#             # Fetch data for each symbol
#             ticker = yf.Ticker(symbol)
#             info = ticker.info
#             history = ticker.history(period="1y")  # Get 1-year historical data

#             # Append the data to the DataFrame
#             final_dataframe = final_dataframe.append(
#                 pd.Series(
#                     [
#                         symbol,
#                         info.get[symbol]['quote']['latestPrice'],  # Latest price
#                         info.get['peRatio'],
#                         "N/A"
#                     ],
#                     index=my_columns
#                 ),
#                 ignore_index=True
#             )

#         except Exception as e:
#             print(f"Error fetching data for {symbol}: {e}")

# # Display the final DataFrame
# print(final_dataframe)
import yfinance as yf
import pandas as pd
import time

# Define a sleep time (in seconds) between API calls
SLEEP_TIME = 0.5  # Adjust as needed based on API limits

# Split the list into chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Read the stock symbols from the CSV file
stocks = pd.read_csv('sp_500_stocks.csv')

# Split the stock tickers into groups of 100
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = [','.join(group) for group in symbol_groups]

# Initialize an empty DataFrame
final_dataframe = pd.DataFrame(columns=my_columns)

# Loop through each batch of symbols
for symbols in symbol_groups:
    for symbol in symbols:
        try:
            # Fetch data for each symbol
            ticker = yf.Ticker(symbol)
            info = ticker.info
            history = ticker.history(period="1y")  # Get 1-year historical data

            # Add a delay to avoid hitting API limits
            time.sleep(SLEEP_TIME)

            # Append the data to the DataFrame
            final_dataframe = final_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        info.get("currentPrice", None),  # Latest price
                        info.get("trailingPE", None),    # P/E Ratio
                        "N/A"
                    ],
                    index=my_columns
                ),
                ignore_index=True
            )

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")

# Display the final DataFrame
print(final_dataframe)


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=nkzFaGiyQCQ
ERROR:yfinance:$ABC: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABMD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABMD&crumb=nkzFaGiyQCQ
ERROR:yfinance:$ABMD: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$ALXN: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$ANTM: possibly delisted; no price data fo

    Ticker   Price Price-to-Earnings Ratio Number of Shares to Buy
0        A  137.87               31.121897                     N/A
1      AAL   17.38                41.38095                     N/A
2      AAP   43.61                55.20253                     N/A
3     AAPL  243.04               39.973682                     N/A
4     ABBV  176.17               61.170135                     N/A
..     ...     ...                     ...                     ...
500    YUM  138.13               25.818693                     N/A
501    ZBH  109.16               20.832062                     N/A
502   ZBRA  404.98                55.85931                     N/A
503   ZION   59.13               13.469249                     N/A
504    ZTS  174.77               32.913372                     N/A

[505 rows x 4 columns]


In [61]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,137.87,31.121897,N/A
1,AAL,17.38,41.38095,N/A
2,AAP,43.61,55.20253,N/A
3,AAPL,243.04,39.973682,N/A
4,ABBV,176.17,61.170135,N/A
...,...,...,...,...
500,YUM,138.13,25.818693,N/A
501,ZBH,109.16,20.832062,N/A
502,ZBRA,404.98,55.85931,N/A
503,ZION,59.13,13.469249,N/A


# Removing Glamour Stocks

Sorted the dataframe such that the stocks with the lowest price earnings ratios were at the top of the data frame, and we specified the in place equals true so that the original frame was actually modified.

Sorted the dataframe such that all rows that have a price to earnings ration val. below zero were dropped , then we trim the dataframe to include the stocks with the 50 lowest price earning ratios, we reset the index, and then drop the new index col. that was created by way of this reset index method.

In [62]:
# Convert 'Price-to-Earnings Ratio' to numeric, replacing non-numeric values with NaN
final_dataframe['Price-to-Earnings Ratio'] = pd.to_numeric(
    final_dataframe['Price-to-Earnings Ratio'], errors='coerce'
)

# Drop rows where 'Price-to-Earnings Ratio' is NaN
final_dataframe.dropna(subset=['Price-to-Earnings Ratio'], inplace=True)

# Filter for rows where 'Price-to-Earnings Ratio' is greater than 0
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]

# Sort the DataFrame by 'Price-to-Earnings Ratio'
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)

final_dataframe = final_dataframe[:50]

# Reset the index and drop the old index
final_dataframe.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,APA,21.84,3.102273,N/A
1,FMC,57.19,4.691551,N/A
2,WU,10.89,5.556122,N/A
3,GM,53.39,5.697972,N/A
4,NOV,15.64,5.792592,N/A
5,APTV,56.91,6.337417,N/A
6,DVN,35.96,6.659259,N/A
7,CE,69.44,6.916335,N/A
8,KSS,15.39,6.932433,N/A
9,CINF,157.16,8.076054,N/A


# Calculating the Number of Shares to Buy

We now need to calculate the number of shares we need to buy.
To fo this, we will use the porfolio_input function that we created in our momentum project.

In [63]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
        portfolio_input()

We use :

```
math.floor
```
because we cant buy fractional share from some broker


In [64]:
postion_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(postion_size/final_dataframe.loc[row, 'Price'])

In [65]:
final_dataframe # Results are not consisten below because, P2E ratio and not correct since in the tutorial while fectching form IEX cloud, AAL was -ve but here its +Ve, basiclay when the company is not in profit and sherding money, i.e., the company is in -ve earning then it becames -ve.

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,APA,21.84,3.102273,2289
1,FMC,57.19,4.691551,874
2,WU,10.89,5.556122,4591
3,GM,53.39,5.697972,936
4,NOV,15.64,5.792592,3196
5,APTV,56.91,6.337417,878
6,DVN,35.96,6.659259,1390
7,CE,69.44,6.916335,720
8,KSS,15.39,6.932433,3248
9,CINF,157.16,8.076054,318


# Building a Better(and more Realistic) Value Strategy

In [66]:
# Symbol for the stock
symbol = 'AAPL'

# Fetch the stock data suing yfinance
ticker = yf.Ticker(symbol)
info = ticker.info

# P/E Ratio
pe_ratio = info.get('trailingPE', None)
print(f"P/E : {pe_ratio}")

# P/B Ratio
pb_ratio = info.get('priceToBook', None)
print(f"P/B : {pb_ratio}")

# P/S Ratio
ps_ratio = info.get('priceToSalesTrailing12Months', None)
print(f"P/S : {ps_ratio}")
SLEEP_TIME = 0.5

#EV/EBITA
try:
    financials = ticker.financials
    ebitda = financials.loc['EBITDA'].iloc[0]  # First reported value for EBITDA
except KeyError:
    ebitda = None

if ebitda is not None:
    enterprise_value = info.get('enterpriseValue', None)
    if enterprise_value is not None:
        ev_to_ebitda = enterprise_value / ebitda
        print(f"EV/EBITDA : {ev_to_ebitda}")
    else:
        print("Enterprise value not available.")
else:
    print("EBITDA not available.")
# enterprise_value = info.get('enterpriseValue', None)
# ebitda = info.get('ebitda', None)
# ev_to_ebitda = enterprise_value/ebitda
# print(f"EV/EBITA : {ev_to_ebitda}")

# Gross Profit (from financial statements)
try:
    financials = ticker.financials
    gross_profit = financials.loc['Gross Profit'].iloc[0]  # First reported value for Gross Profit
except KeyError:
    gross_profit = None

if enterprise_value is not None and gross_profit is not None:
    ev_to_gross_profit = enterprise_value / gross_profit
else:
    ev_to_gross_profit = None
print(f"EV/GP: {ev_to_gross_profit}")

P/E : 39.973682
P/B : 64.51818
P/S : 9.394924
EV/EBITDA : 27.67827434354416
EV/GP: 20.628305382221903


>rv = robust value, equivalent to high value monetum

In [67]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the columns for the dataframe
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]
SLEEP_TIME = 0.5


# Create an empty DataFrame
rv_dataframe = pd.DataFrame(columns=rv_columns)

# Iterate over symbol strings
for symbol_string in symbol_strings:  # Assuming symbol_strings is defined
    for symbol in symbol_string.split(','):
        try:
            # Fetch data from yfinance
            ticker = yf.Ticker(symbol)
            time.sleep(SLEEP_TIME)
            info = ticker.info

            # Retrieve required data fields
            price = info.get('currentPrice', np.NaN)
            pe_ratio = info.get('trailingPE', np.NaN)
            pb_ratio = info.get('priceToBook', np.NaN)
            ps_ratio = info.get('priceToSalesTrailing12Months', np.NaN)
            enterprise_value = info.get('enterpriseValue', np.NaN)
            ebitda = info.get('ebitda', np.NaN)

            # Gross Profit from financial statements
            try:
                financials = ticker.financials
                gross_profit = financials.loc['Gross Profit'].iloc[0]
            except (KeyError, IndexError):
                gross_profit = np.NaN

            # Calculate metrics
            ev_to_ebitda = enterprise_value / ebitda if ebitda else np.NaN
            ev_to_gross_profit = enterprise_value / gross_profit if gross_profit else np.NaN

            # Append data to the DataFrame
            rv_dataframe = rv_dataframe.append(
                pd.Series([
                    symbol,
                    price,
                    'N/A',  # Placeholder for 'Number of Shares to Buy'
                    pe_ratio,
                    'N/A',  # Placeholder for PE Percentile
                    pb_ratio,
                    'N/A',  # Placeholder for PB Percentile
                    ps_ratio,
                    'N/A',  # Placeholder for PS Percentile
                    ev_to_ebitda,
                    'N/A',  # Placeholder for EV/EBITDA Percentile
                    ev_to_gross_profit,
                    'N/A',  # Placeholder for EV/GP Percentile
                    'N/A'  # Placeholder for RV Score
                ], index=rv_columns),
                ignore_index=True
            )

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")

# Display the final DataFrame
print(rv_dataframe)


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=nkzFaGiyQCQ
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABMD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABMD&crumb=nkzFaGiyQCQ
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=nkzFaGiyQCQ
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DISH?modules=financialData%2CquoteType%2CdefaultKeyStatist

    Ticker   Price Number of Shares to Buy Price-to-Earnings Ratio  \
0        A  137.87                     N/A               31.621557   
1      AAL   17.38                     N/A                41.38095   
2      AAP   43.61                     N/A                55.91026   
3     AAPL  243.04                     N/A               39.973682   
4     ABBV  176.17                     N/A               61.170135   
..     ...     ...                     ...                     ...   
500    YUM  138.13                     N/A               25.818693   
501    ZBH  109.16                     N/A               20.832062   
502   ZBRA  404.98                     N/A                55.85931   
503   ZION   59.13                     N/A               13.469249   
504    ZTS  174.77                     N/A                32.97547   

    PE Percentile  Price-to-Book Ratio PB Percentile  Price-to-Sales Ratio  \
0             N/A             6.715537           N/A              6.051956   
1  

In [68]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.87,N/A,31.621557,N/A,6.715537,N/A,6.051956,N/A,25.562468,N/A,11.538153,N/A,N/A
1,AAL,17.38,N/A,41.38095,N/A,NaN,N/A,0.213026,N/A,7.894437,N/A,3.387054,N/A,N/A
2,AAP,43.61,N/A,55.91026,N/A,1.029995,N/A,0.231080,N/A,17.933747,N/A,1.423975,N/A,N/A
3,AAPL,243.04,N/A,39.973682,N/A,64.518180,N/A,9.394924,N/A,27.678275,N/A,20.628305,N/A,N/A
4,ABBV,176.17,N/A,61.170135,N/A,51.617344,N/A,5.605983,N/A,14.666663,N/A,11.087708,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.13,N/A,25.818693,N/A,NaN,N/A,5.338018,N/A,19.195243,N/A,14.319563,N/A,N/A
501,ZBH,109.16,N/A,20.832062,N/A,1.760730,N/A,2.861025,N/A,10.987617,N/A,5.291453,N/A,N/A
502,ZBRA,404.98,N/A,55.85931,N/A,6.101577,N/A,4.563549,N/A,28.124383,N/A,10.637720,N/A,N/A
503,ZION,59.13,N/A,13.469249,N/A,1.469032,N/A,2.903676,N/A,NaN,N/A,NaN,N/A,N/A


# Dealing with Missing Data in Our DataFrame

In [69]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
1,AAL,17.38,N/A,41.38095,N/A,NaN,N/A,0.213026,N/A,7.894437,N/A,3.387054,N/A,N/A
5,ABC,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
6,ABMD,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
13,ADSK,305.17,N/A,60.549606,N/A,35.206510,N/A,11.329350,N/A,49.488823,N/A,NaN,N/A,N/A
17,AFL,107.75,N/A,15.89233,N/A,2.415865,N/A,3.459608,N/A,13.123813,N/A,NaN,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,XLNX,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
497,XRAY,19.31,N/A,NaN,N/A,1.541101,N/A,0.991861,N/A,9.746987,N/A,2.817562,N/A,N/A
498,XRX,8.71,N/A,NaN,N/A,0.831345,N/A,0.171543,N/A,12.429027,N/A,1.831590,N/A,N/A
500,YUM,138.13,N/A,25.818693,N/A,NaN,N/A,5.338018,N/A,19.195243,N/A,14.319563,N/A,N/A


Dealing with missing data is important in data science

Two main approaches to it:


1.   Drop missing data from the data set(pandas' `dropna`)




2.   Replace missing data with new value(panda's `fillna` method is useful here)




In [73]:
# Ensure columns are numeric
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column] = pd.to_numeric(rv_dataframe[column], errors='coerce')  # Convert to numeric, coerce errors to NaN
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)  # Fill NaN with column mean

# Display the updated DataFrame
rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.87,N/A,31.621557,N/A,6.715537,N/A,6.051956,N/A,25.562468,N/A,11.538153,N/A,N/A
1,AAL,17.38,N/A,41.380950,N/A,8.433178,N/A,0.213026,N/A,7.894437,N/A,3.387054,N/A,N/A
2,AAP,43.61,N/A,55.910260,N/A,1.029995,N/A,0.231080,N/A,17.933747,N/A,1.423975,N/A,N/A
3,AAPL,243.04,N/A,39.973682,N/A,64.518180,N/A,9.394924,N/A,27.678275,N/A,20.628305,N/A,N/A
4,ABBV,176.17,N/A,61.170135,N/A,51.617344,N/A,5.605983,N/A,14.666663,N/A,11.087708,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.13,N/A,25.818693,N/A,8.433178,N/A,5.338018,N/A,19.195243,N/A,14.319563,N/A,N/A
501,ZBH,109.16,N/A,20.832062,N/A,1.760730,N/A,2.861025,N/A,10.987617,N/A,5.291453,N/A,N/A
502,ZBRA,404.98,N/A,55.859310,N/A,6.101577,N/A,4.563549,N/A,28.124383,N/A,10.637720,N/A,N/A
503,ZION,59.13,N/A,13.469249,N/A,1.469032,N/A,2.903676,N/A,15.279801,N/A,11.505488,N/A,N/A


# Calculating Value Percentiles



*   Price-to-earning rato
*   Price-to-book ratio
*   Price-to-sales ratio
*   EV/EBITA
*   EV/GP






In [74]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile',
}

for metric in metrics.keys():
  for row in rv_dataframe.index:
    rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

In [75]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.87,N/A,31.621557,0.538614,6.715537,0.621782,6.051956,0.807921,25.562468,0.893069,11.538153,0.665347,N/A
1,AAL,17.38,N/A,41.380950,0.661386,8.433178,0.754455,0.213026,0.013861,7.894437,0.09505,3.387054,0.061386,N/A
2,AAP,43.61,N/A,55.910260,0.754455,1.029995,0.037624,0.231080,0.017822,17.933747,0.669307,1.423975,0.005941,N/A
3,AAPL,243.04,N/A,39.973682,0.647525,64.518180,0.984158,9.394924,0.90297,27.678275,0.910891,20.628305,0.928713,N/A
4,ABBV,176.17,N/A,61.170135,0.776238,51.617344,0.978218,5.605983,0.772277,14.666663,0.388119,11.087708,0.441584,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.13,N/A,25.818693,0.435644,8.433178,0.754455,5.338018,0.750495,19.195243,0.716832,14.319563,0.792079,N/A
501,ZBH,109.16,N/A,20.832062,0.30099,1.760730,0.176238,2.861025,0.433663,10.987617,0.211881,5.291453,0.148515,N/A
502,ZBRA,404.98,N/A,55.859310,0.752475,6.101577,0.6,4.563549,0.720792,28.124383,0.920792,10.637720,0.423762,N/A
503,ZION,59.13,N/A,13.469249,0.10297,1.469032,0.10297,2.903676,0.443564,15.279801,0.487129,11.505488,0.560396,N/A


# Calculating the RV Score
RV Score will be the arthmetic mean of the 4 percentile scores that we calculated in the last section.

In [76]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.87,N/A,31.621557,0.538614,6.715537,0.621782,6.051956,0.807921,25.562468,0.893069,11.538153,0.665347,0.705347
1,AAL,17.38,N/A,41.380950,0.661386,8.433178,0.754455,0.213026,0.013861,7.894437,0.09505,3.387054,0.061386,0.317228
2,AAP,43.61,N/A,55.910260,0.754455,1.029995,0.037624,0.231080,0.017822,17.933747,0.669307,1.423975,0.005941,0.29703
3,AAPL,243.04,N/A,39.973682,0.647525,64.518180,0.984158,9.394924,0.90297,27.678275,0.910891,20.628305,0.928713,0.874851
4,ABBV,176.17,N/A,61.170135,0.776238,51.617344,0.978218,5.605983,0.772277,14.666663,0.388119,11.087708,0.441584,0.671287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.13,N/A,25.818693,0.435644,8.433178,0.754455,5.338018,0.750495,19.195243,0.716832,14.319563,0.792079,0.689901
501,ZBH,109.16,N/A,20.832062,0.30099,1.760730,0.176238,2.861025,0.433663,10.987617,0.211881,5.291453,0.148515,0.254257
502,ZBRA,404.98,N/A,55.859310,0.752475,6.101577,0.6,4.563549,0.720792,28.124383,0.920792,10.637720,0.423762,0.683564
503,ZION,59.13,N/A,13.469249,0.10297,1.469032,0.10297,2.903676,0.443564,15.279801,0.487129,11.505488,0.560396,0.339406


# Selecting 50 Best Value Stocks

In [77]:
rv_dataframe.sort_values(by = 'RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

In [78]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,15.39,N/A,6.901345,0.015842,0.438812,0.00198,0.099974,0.00396,6.558696,0.043564,1.330913,0.00198,0.013465
1,CNC,57.95,N/A,10.311388,0.041584,1.082469,0.047525,0.200110,0.011881,5.121436,0.017822,1.750724,0.011881,0.026139
2,BWA,33.99,N/A,8.605063,0.023762,1.207546,0.055446,0.524639,0.057426,5.603484,0.029703,4.106603,0.09505,0.052277
3,PVH,108.86,N/A,9.026534,0.029703,1.225639,0.059406,0.709448,0.089109,7.644928,0.075248,1.725925,0.009901,0.052673
4,NOV,15.64,N/A,5.792592,0.009901,0.941431,0.029703,0.683355,0.081188,6.905967,0.051485,4.163171,0.09703,0.053861
5,APA,21.84,N/A,3.151515,0.00198,1.579747,0.136634,0.902249,0.128713,3.103791,0.009901,3.705899,0.075248,0.070495
6,VLO,133.05,N/A,11.795214,0.071287,1.667983,0.156436,0.329715,0.031683,6.459911,0.039604,3.917345,0.081188,0.07604
7,CVS,56.86,N/A,14.730571,0.144554,0.954443,0.033663,0.194837,0.009901,10.360366,0.180198,2.686641,0.033663,0.080396
8,MHK,131.43,N/A,14.817361,0.148515,1.056528,0.041584,0.767297,0.10495,6.748276,0.047525,3.858898,0.077228,0.08396
9,APTV,56.91,N/A,6.281456,0.011881,1.505675,0.114851,0.678119,0.079208,7.192999,0.057426,6.343811,0.20198,0.093069


# Calculating the number of Shares to Buy

In [83]:
portfolio_input()

Enter the value of your portfolio:17000000


In [84]:
print(portfolio_size)

17000000


In [85]:
position_size = float(portfolio_size)/len(rv_dataframe.index)


for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
rv_dataframe

/usr/local/lib/python3.10/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,15.39,22092,6.901345,0.015842,0.438812,0.00198,0.099974,0.00396,6.558696,0.043564,1.330913,0.00198,0.013465
1,CNC,57.95,5867,10.311388,0.041584,1.082469,0.047525,0.200110,0.011881,5.121436,0.017822,1.750724,0.011881,0.026139
2,BWA,33.99,10002,8.605063,0.023762,1.207546,0.055446,0.524639,0.057426,5.603484,0.029703,4.106603,0.09505,0.052277
3,PVH,108.86,3123,9.026534,0.029703,1.225639,0.059406,0.709448,0.089109,7.644928,0.075248,1.725925,0.009901,0.052673
4,NOV,15.64,21739,5.792592,0.009901,0.941431,0.029703,0.683355,0.081188,6.905967,0.051485,4.163171,0.09703,0.053861
5,APA,21.84,15567,3.151515,0.00198,1.579747,0.136634,0.902249,0.128713,3.103791,0.009901,3.705899,0.075248,0.070495
6,VLO,133.05,2555,11.795214,0.071287,1.667983,0.156436,0.329715,0.031683,6.459911,0.039604,3.917345,0.081188,0.07604
7,CVS,56.86,5979,14.730571,0.144554,0.954443,0.033663,0.194837,0.009901,10.360366,0.180198,2.686641,0.033663,0.080396
8,MHK,131.43,2586,14.817361,0.148515,1.056528,0.041584,0.767297,0.10495,6.748276,0.047525,3.858898,0.077228,0.08396
9,APTV,56.91,5974,6.281456,0.011881,1.505675,0.114851,0.678119,0.079208,7.192999,0.057426,6.343811,0.20198,0.093069


# Formatting Our Excel

In [86]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx Fiel

In [87]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [88]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

# Saving Our Excel Output


In [89]:
writer.save()